In [ ]:
import pandas as pd
import numpy as np
from poker_utils.constants import HANDS_DICT

In [ ]:
def process_base_hand_data(path_to_data: str) -> pd.DataFrame:
    raw_hand_data = pd.read_csv(path_to_data)
    raw_hand_data = raw_hand_data.rename(columns={
        'win_perc.flop': 'flop_wp',
        'win_perc.turn': 'turn_wp',
        'win_perc.river': 'river_wp'})
    raw_hand_data = raw_hand_data.assign(
        delta_wp=raw_hand_data['river_wp']-raw_hand_data['flop_wp']
        )
    raw_hand_data.set_index("hand_idx", inplace=True)
    raw_hand_data.index.name = "hand"
    return raw_hand_data[
        ['connectedness','suited','pair','flop_wp','turn_wp','river_wp','delta_wp']
        ]

    

In [16]:
def process_dom_block_data(path_to_data: str) -> pd.DataFrame:
    raw_data = pd.read_csv(path_to_data)
    raw_data['hand'] = raw_data['hand'].apply(lambda x: {j:i for i,j in HANDS_DICT.items()}[x])
    raw_data.set_index('hand', inplace=True)
    
    return raw_data[
    ['combos_dominated', 'dom_ratio', 'block_ratio_cat', 'dom_block_ratio', 'cluster']
    ]
    
    

In [ ]:
# def process_hand_ev_data(path_to_data: str) -> pd.DataFrame:
#     raw_data = pd.read_excel(path_to_data)
#     raw_data['hand'] = raw_data['hand'].apply(lambda x: {j:i for i,j in HANDS_DICT.items()}[x])
    